<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/llama_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune Llama on Colab TPU

This example shows to tune one of Meta's Llama models on single host TPUs. For information on TPUs architecture, you can consult the [documentation](https://cloud.google.com/tpu/docs/system-architecture-tpu-vm).


In [ ]:
!git clone https://github.com/huggingface/optimum-tpu.git

# Install Optimum TPU
%cd optimum-tpu
!pip install -e . -f https://storage.googleapis.com/libtpu-releases/index.html -q

# Install TRL and PEFT for training (see later how they are used)
!pip install trl peft -q

# This will be necessary for the language modeling example
!pip install datasets evaluate accelerate -q

Then, the tokenizer and model need to be loaded. We will choose [`meta-llama/Llama-3.2-1B`](https://huggingface.co/meta-llama/Llama-3.2-1B) for this example.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Add custom token for padding Llama
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)

/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops9from_file4callESt17basic_string_viewIcSt11char_traitsIcEESt8optionalIbES6_IlES6_IN3c1010ScalarTypeEES6_INS9_6LayoutEES6_INS9_6DeviceEES7_'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


To tune the model with the [Abirate/english_quotes](https://huggingface.co/datasets/Abirate/english_quotes) dataset, you can load it and obtain the `quote` column:


In [3]:
from datasets import load_dataset


data = load_dataset("Abirate/english_quotes")

def preprocess_function(samples):
    # Create prompt, completion, and combined text columns
    prompts = [f"Generate a quote:\n\n" for _ in samples["quote"]]
    completions = [f"{quote}{tokenizer.eos_token}" for quote in samples["quote"]]
    texts = [p + c for p, c in zip(prompts, completions)]
    return {"prompt": prompts, "completion": completions, "text": texts}

# data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
data = data.map(
    preprocess_function,
    batched=True,
    remove_columns=data["train"].column_names
)

You then need to specify the FSDP training arguments to enable the sharding feature, the function will deduce the classes that should be sharded:


In [4]:
from optimum.tpu import fsdp_v2

fsdp_training_args = fsdp_v2.get_fsdp_training_args(model)

The `fsdp_training_args` will specify the Pytorch module that needs to be sharded:

In [5]:
fsdp_training_args

{'fsdp': 'full_shard',
 'fsdp_config': {'transformer_layer_cls_to_wrap': ['LlamaDecoderLayer'],
  'xla': True,
  'xla_fsdp_v2': True,
  'xla_fsdp_grad_ckpt': True}}

Now training can be done as simply as using the standard `Trainer` class:

In [6]:
from peft import LoraConfig


lora_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig
from peft import PeftModel


def formatting_func(examples):
    # The 'prompt' column already contains the combined prompt and completion
    return examples["prompt"]

# Create a copy to avoid modifying the original dictionary
fsdp_training_args_copy = fsdp_training_args.copy()

# Extract fsdp_config from the copy
fsdp_config_value = fsdp_training_args_copy.get('fsdp_config', None)

# Remove keys that are explicitly passed to SFTConfig from the copy
if 'fsdp' in fsdp_training_args_copy:
    del fsdp_training_args_copy['fsdp']
if 'fsdp_config' in fsdp_training_args_copy:
    del fsdp_training_args_copy['fsdp_config']

# Check if the model is already a PeftModel and unload if necessary
if isinstance(model, PeftModel):
    print("Unloading existing PEFT adapters...")
    model = model.unload()


sft_config = SFTConfig(
    per_device_train_batch_size=32,
    num_train_epochs=10,
    max_steps=-1,
    output_dir="/content/output",
    optim="adafactor",
    logging_steps=1,
    dataloader_drop_last=True,  # Required by FSDP v2
    completion_only_loss=False, # Disable completion_only_loss
    fsdp="full_shard", # Explicitly set fsdp to "full_shard"
    fsdp_config=fsdp_config_value, # Pass fsdp_config dictionary directly
    **fsdp_training_args_copy, # Unpack remaining args from the copy
)


trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=sft_config, # Pass the SFTConfig object here
    peft_config=lora_config,
    #formatting_func=formatting_func, # Remove formatting_func when using SFTConfig
    #max_seq_length=512,
    #packing=True,
)

trainer.train()